In [3]:
import os 
%pwd

'd:\\WineQuality_EndToEndDataScienceProject\\WineQualityDataScienceProject\\research'

In [4]:
os.chdir("../")
%pwd

'd:\\WineQuality_EndToEndDataScienceProject\\WineQualityDataScienceProject'

In [8]:
# This will be put in src/WineQualityPrediction/config/configuration.py
from src.WineQualityPrediction.constants import config_file_path , params_file_path , schema_file_path
from src.WineQualityPrediction.utils.common import read_yaml , create_directories
from src.WineQualityPrediction.entity.config_entity import DataIngestionConfig

class ConfigurationManager:
    def __init__(self , config_file_path = config_file_path ,
                 params_file_path = params_file_path ,
                   schema_file_path = schema_file_path):
        
        self.config = read_yaml(config_file_path)
        self.params = read_yaml(params_file_path)
        self.schema = read_yaml(schema_file_path)

        create_directories([self.config.artifacts_root])

    
    def get_data_ingestion_config(self) -> DataIngestionConfig :
        config = self.config.data_ingestion
        create_directories([config.root_dir])
        data_ingestion_config = DataIngestionConfig(
            root_dir= config.root_dir,
            source_url= config.source_url,
            local_data_file=config.datafileLocation,
            unzip_dir= config.unzip_dir

        )
        return data_ingestion_config




In [20]:
import zipfile 
import urllib.request as request 
import os 
from src.WineQualityPrediction import logger 
from src.WineQualityPrediction.entity.config_entity import DataIngestionConfig




class DataIngestion:
    def __init__(self , config : DataIngestionConfig):
        self.config = config 
    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename , headers = request.urlretrieve(url = self.config.source_url ,
                                                     filename = self.config.local_data_file)
            logger.info(f"{filename} downloaded with following info : \n {headers}")

        else :
            logger.info(f"File already exists")

    def extract_zip_file(self):
        unzip_path = self.config.unzip_dir
        with zipfile.ZipFile(self.config.local_data_file , 'r') as file:
            file.extractall(unzip_path)
        logger.info(f"{self.config.local_data_file} extracted at {unzip_path}")
        

        

In [21]:
try :
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e :
    raise e




[2025-04-10 14:09:20,236] : INFO : common : yaml file: config\config.yml loaded successfully
[2025-04-10 14:09:20,239] : INFO : common : yaml file: params.yml loaded successfully
[2025-04-10 14:09:20,241] : INFO : common : yaml file: schema.yml loaded successfully
[2025-04-10 14:09:20,242] : INFO : common : Created directory at: artifacts
[2025-04-10 14:09:20,243] : INFO : common : Created directory at: artifacts/data_ingestion
[2025-04-10 14:09:20,243] : INFO : 3063299882 : File already exists
[2025-04-10 14:09:20,250] : INFO : 3063299882 : artifacts/data_ingestion/data.zip extracted at artifacts/data_ingestion
